## **Description du corpus**
- Nombre de sites distincts qui ont été aspirés
- Nombre de documents aspirés / Nombre moyen de documents par site, minimum, maximum
- Nombre de types/tokens / Nombre moyen de types/tokens par site, minimum, maximum

*Nombre de sites distincts*  
Pour chaque acteur, ouvrir les CSV contenanant la liste des URLs et compter les noms de domaines distincts;  
Pour ce faire, splitter l'URL et retenir le nom de domaine 

#### **Acteurs du système de santé dont les sites Web nous intéressent pour la constitution du corpus**

In [ ]:
from pandas import *
from pathlib import Path
import os, re, shutil, nltk

path = '../03-corpus/2-data/1-fr/'


nb = len([file for file in os.listdir() if file.endswith('csv') and not 'chum' in file]) # (le chum se trouve déjà dans chu_iu)
print('Les sites Web de {} acteurs ont été aspirés pour constituer notre corpus :'.format(nb))
for file in os.listdir(path):
    if file.endswith('csv') and not 'chum' in file:
        print(file[:-4])

#### **Nombre de sites aspirés**

In [ ]:
nb_domaines = 0
domaines = []

for file in os.listdir(path):
    if file.endswith('csv') and not 'chum' in file:
        # encoding= 'ISO-8859-1' "utf-8"
        # sep = ';' ','
        try:
            with open(path + file, encoding = 'ISO-8859-1') as f:
                csv = read_csv(f, sep=';')

        except: 
            try: 
                with open(path + file, encoding = 'ISO-8859-1') as f:
                    csv = read_csv(f, sep=',')
            
            except: 
                try:
                    with open(path + file, encoding = 'utf-8') as f:
                        csv = read_csv(f, sep= ',')

                except: 
                    try:
                        with open(path + file, encoding = 'utf-8') as f:
                            csv = read_csv(f, sep= ';')

                    except Exception as e:
                        print(file + '-' + str(e))

        urls = csv['url'].tolist()
        regex = 'https?://(www)?1?\.?(\w+)'
        try:
            domaines += [re.match(regex, url).group(2) for url in urls]
        except Exception as e:
            True
            #print(file + '-' + str(e))

In [ ]:
domaines += ['ramq', 'sante_mtl', 'urgence_sante']
domaines = set(domaines)
nb_domaines = len(domaines) + 22 # (cisss/ciusss)

In [ ]:
print('{} sites distincts ont été aspirés au total'.format(nb_domaines))

#### **Nombre de documents** (fr)
#### **Nombre de types / tokens** (fr)

In [ ]:
from nltk.tokenize import RegexpTokenizer
# Seulement les caractères alphabétiques
tokenizer_re = RegexpTokenizer(r"\w\'|\w+")

nb_docs_fr = 0
corpus = []
fichiers = [f for f in os.listdir(path) if f.endswith('.csv') and not (f == 'mf.csv') and not (f == 'chum.csv')]
print('Voici le nombre de documents (HTML ou PDF) aspirés pour chacun des ' + str(len(fichiers)) + ' acteurs considérés.\n(Noter que le chum devrait se trouver sous CHU_IU)')

for file in fichiers:
    nb_tokens = 0
    nb_types = set()
    with open(path + file, encoding = 'ISO-8859-1') as f:
        data = read_csv(f)['text'].tolist()
        nb_doc = len(data)
        subcorpus = [re.sub('\d', '', t.strip('\n').lower().replace('’', '\'').replace('  ', ' ')) for t in data]
        corpus.append(subcorpus)
        tokens = nltk.flatten([tokenizer_re.tokenize(doc) for doc in subcorpus])
        types = set(tokens)

        print(file + '\t' + str(nb_doc) + ' documents\t' + str(len(tokens)) + ' tokens\t' + str(len(types)) +' formes uniques')

        nb_docs_fr += nb_doc

print('\nAu total, notre corpus francophone compte donc un total de {} documents.'.format(nb_docs_fr)+'\nOn a donc un une moyenne d\'environ {} documents par site (mais ça varie considérablement d\'un site à l\'autre).'.format(round(nb_docs_fr/157)))


#### **Déterminer la taille d'échantillon à retenir**

In [ ]:
x = []
for i in range(12): # On va aller jusqu'à max 115 000 documents (on en a 115 080)
    a = 10000
    x.append(a * i)

In [ ]:
corpus = nltk.flatten(corpus)

In [ ]:
len(corpus)

In [ ]:
import random
random.shuffle(corpus)

y = []
types = set()

for valeur in x:
    if (len(corpus) > valeur):
        tokens = nltk.flatten([tokenizer_re.tokenize(doc) for doc in corpus[:valeur]])
        tokens =  set(tokens)
        types.update(tokens)
        y.append(len(types))
        print("Nombre de documents considérés : " + str(valeur) + " - " + str(len(types)) + " formes uniques.")

In [ ]:
y

In [ ]:
x = x[:-1]

In [ ]:
# Refaire en randomisant l'ordre des documents

import matplotlib.pyplot as plt
plt.plot(x, y)
plt.xlabel('Nombre de documents')
plt.ylabel('Nombre de formes uniques')
plt.show()